In [11]:
import pandas as pd

In [45]:
#Importing the two dataframes CSV files
stats2018 = pd.read_csv('Data/2017-2018stats.txt')
salaries2019 = pd.read_csv('Data/2018-2019salaries.txt')

In [46]:
#Removing the string at the end of the player's name and extracting the columns I deem important
stats2018[['Name','string']] = stats2018['Player'].str.split('\\', expand = True)
stats2018 = stats2018[['Name','Pos','Tm','G','GS', 'FG', 'FG%', '3P', '3P%', 'eFG%', 'FT%', 'ORB','TRB','AST','STL','BLK','PS/G']]
stats2018.head()

,Name,Pos,Tm,G,GS,FG,FG%,3P,3P%,eFG%,FT%,ORB,TRB,AST,STL,BLK,PS/G
0,Alex Abrines,SG,OKC,75,8,1.5,0.395,1.1,0.380,0.540,0.848,0.3,1.5,0.4,0.5,0.1,4.7
1,Quincy Acy,PF,BRK,70,8,1.9,0.356,1.5,0.349,0.496,0.817,0.6,3.7,0.8,0.5,0.4,5.9
2,Steven Adams,C,OKC,76,76,5.9,0.629,0.0,0.000,0.629,0.557,5.1,9.0,1.2,1.2,1.0,13.9
3,Bam Adebayo,C,MIA,69,19,2.5,0.512,0.0,0.000,0.512,0.721,1.7,5.5,1.5,0.5,0.6,6.9
4,Arron Afflalo,SG,ORL,53,3,1.2,0.401,0.5,0.386,0.485,0.846,0.1,1.2,0.6,0.1,0.2,3.4


In [47]:
stats2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664 entries, 0 to 663
Data columns (total 17 columns):
Name    664 non-null object
Pos     664 non-null object
Tm      664 non-null object
G       664 non-null int64
GS      664 non-null int64
FG      664 non-null float64
FG%     660 non-null float64
3P      664 non-null float64
3P%     599 non-null float64
eFG%    660 non-null float64
FT%     606 non-null float64
ORB     664 non-null float64
TRB     664 non-null float64
AST     664 non-null float64
STL     664 non-null float64
BLK     664 non-null float64
PS/G    664 non-null float64
dtypes: float64(12), int64(2), object(3)
memory usage: 93.4+ KB


In [48]:
#Checking to see which players played for multiple teams 
multiple_teams = stats2018['Name'].loc[stats2018['Tm'] == 'TOT']
multiple_teams.count()



    



59

In [49]:
#Counting up all of the times a person who has played for multiple teams shows up in the dataframe
count_multiple = sum(stats2018['Name'].isin(multiple_teams))
count_multiple

183

In [50]:
#Subtracting away the amount of times the TOT shows up from the total
count_multiple - len(multiple_teams)

124

In [51]:
#Dropping the duplicates caused by a player changing teams midway, keeping the TOT entry. The amount makes sense as 660 - 124 = 540
stats2018u = stats2018.drop_duplicates(subset = ['Name'])
stats2018u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 0 to 663
Data columns (total 17 columns):
Name    540 non-null object
Pos     540 non-null object
Tm      540 non-null object
G       540 non-null int64
GS      540 non-null int64
FG      540 non-null float64
FG%     537 non-null float64
3P      540 non-null float64
3P%     491 non-null float64
eFG%    537 non-null float64
FT%     501 non-null float64
ORB     540 non-null float64
TRB     540 non-null float64
AST     540 non-null float64
STL     540 non-null float64
BLK     540 non-null float64
PS/G    540 non-null float64
dtypes: float64(12), int64(2), object(3)
memory usage: 75.9+ KB


In [57]:
#Splitting the weird Player column, and dropping all unneccessary columns
salaries2019[['Name','string']] = salaries2019['Player'].str.split('\\', expand = True)
dropped2019 = salaries2019.drop(['Player','2019-20','2020-21','2021-22','2022-23','2023-24', 'Signed Using','string'],1)
dropped2019.head()

,﻿Rk,Tm,2018-19,Guaranteed,Name
0,1,GSW,$37457154,$166476240,Stephen Curry
1,2,LAL,$35654150,$226621146,LeBron James
2,3,LAL,$35654150,$226621146,LeBron James
3,4,HOU,$35654150,$159730592,Chris Paul
4,5,OKC,$35350000,$158382000,Russell Westbrook


In [58]:
#Counting to see how many nonunique entries there are. There are 450 unique entries in the name column and 2 that show up twice
len(dropped2019['Name']) - dropped2019['Name'].nunique()


2

In [71]:
#Counting to see how many entries there are
dropped2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 0 to 451
Data columns (total 5 columns):
﻿Rk           452 non-null int64
Tm            452 non-null object
2018-19       452 non-null object
Guaranteed    425 non-null object
Name          452 non-null object
dtypes: int64(1), object(4)
memory usage: 21.2+ KB


In [72]:
#dropping duplicates that show up in the salary column for some reason. should be -2. This is correct
dropped2019u = dropped2019.drop_duplicates(subset = ['Name'])
dropped2019u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 451
Data columns (total 5 columns):
﻿Rk           450 non-null int64
Tm            450 non-null object
2018-19       450 non-null object
Guaranteed    423 non-null object
Name          450 non-null object
dtypes: int64(1), object(4)
memory usage: 21.1+ KB


In [97]:
#Concatenating the two data frames together. Checking, we now only have 289 entries, because some players that played 
#last year do not have salaries for next year, and some who have salaries for next year did not play last year
data2018 = pd.merge(dropped2019u,stats2018ud, how = 'inner', on = ['Name', 'Tm'])
data2018.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 288
Data columns (total 20 columns):
﻿Rk           289 non-null int64
Tm            289 non-null object
2018-19       289 non-null object
Guaranteed    269 non-null object
Name          289 non-null object
Pos           289 non-null object
G             289 non-null int64
GS            289 non-null int64
FG            289 non-null float64
FG%           288 non-null float64
3P            289 non-null float64
3P%           271 non-null float64
eFG%          288 non-null float64
FT%           283 non-null float64
ORB           289 non-null float64
TRB           289 non-null float64
AST           289 non-null float64
STL           289 non-null float64
BLK           289 non-null float64
PS/G          289 non-null float64
dtypes: float64(12), int64(3), object(5)
memory usage: 47.4+ KB


In [98]:
#Taking the dollar sign out of the salary and casting it as an integer, so that I can order by it. Checking to see 
data2018['2018-19'] = data2018['2018-19'].str.replace('$', '').astype(int)
data2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 288
Data columns (total 20 columns):
﻿Rk           289 non-null int64
Tm            289 non-null object
2018-19       289 non-null int32
Guaranteed    269 non-null object
Name          289 non-null object
Pos           289 non-null object
G             289 non-null int64
GS            289 non-null int64
FG            289 non-null float64
FG%           288 non-null float64
3P            289 non-null float64
3P%           271 non-null float64
eFG%          288 non-null float64
FT%           283 non-null float64
ORB           289 non-null float64
TRB           289 non-null float64
AST           289 non-null float64
STL           289 non-null float64
BLK           289 non-null float64
PS/G          289 non-null float64
dtypes: float64(12), int32(1), int64(3), object(4)
memory usage: 46.3+ KB


In [92]:
#Sorting by highest salary first and checking how it looks
data2018.sort_values(by = '2018-19', ascending = False)
data2018.head()


,﻿Rk,Tm,2018-19,Guaranteed,Name,Pos,G,GS,FG,FG%,3P,3P%,eFG%,FT%,ORB,TRB,AST,STL,BLK,PS/G
0,1,GSW,37457154,$166476240,Stephen Curry,PG,51,51,8.4,0.495,4.2,0.423,0.618,0.921,0.7,5.1,6.1,1.6,0.2,26.4
1,4,HOU,35654150,$159730592,Chris Paul,PG,58,58,6.3,0.460,2.5,0.380,0.550,0.919,0.6,5.4,7.9,1.7,0.2,18.6
2,5,OKC,35350000,$158382000,Russell Westbrook,PG,80,80,9.5,0.449,1.2,0.298,0.477,0.737,1.9,10.1,10.3,1.8,0.3,25.4
3,7,BOS,31214295,$63914985,Gordon Hayward,SF,1,1,1.0,0.500,0.0,0.000,0.500,NaN,0.0,1.0,0.0,0.0,0.0,2.0
4,8,TOR,31000000,$64296296,Kyle Lowry,PG,78,78,5.2,0.427,3.1,0.399,0.553,0.854,0.8,5.6,6.9,1.1,0.2,16.2
